# Perform regionalization when no parameter set is available

Here we call the Regionalization WPS service to provide estimated streamflow (best estimate and ensemble) at an ungauged site using three pre-calibrated hydrological models and a large hydrometeorological database with catchment attributes (Extended CANOPEX). Multiple regionalization strategies are allowed. 

In [ ]:
import datetime as dt

import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

from ravenpy import Emulator
from ravenpy.new_config import commands as rc
from ravenpy.new_config.emulators import GR4JCN
from ravenpy.utilities.new_config.regionalization import regionalize
from ravenpy.utilities.regionalization import read_gauged_params, read_gauged_properties

In [ ]:
%%capture --no-display
ts = "ERA5_weather_data_Salmon.nc"

# Set alternative names for netCDF variables
alt_names = {
    "TEMP_MIN": "tmin",
    "TEMP_MAX": "tmax",
    "PRECIP": "pr",
}

# Data types to extract from netCDF
data_type = ["TEMP_MAX", "TEMP_MIN", "PRECIP"]


hru = {}
hru = dict(
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    hru_type="land",
)


# Model configuration
model_config = GR4JCN(
    Gauge=[
        rc.Gauge.from_nc(
            ts,
            data_type=data_type,
            alt_names=alt_names,
            extra={
                "ALL": {
                    "elevation": hru["elevation"],
                    "Latitude": hru["latitude"],
                    "Longitude": hru["longitude"],
                },
            },
        )
    ],
    HRUs=[hru],
    StartDate=dt.datetime(1990, 1, 1),
    EndDate=dt.datetime(2010, 12, 31),
    RunName="regionalization",
    GlobalParameter={"AVG_ANNUAL_RUNOFF": 208.480},
)

model_structure = "GR4JCN"

nash, params = read_gauged_params(model_structure)

variables = ["latitude", "longitude", "area", "forest"]

props = read_gauged_properties(variables)

ungauged_props = {
    "latitude": 40.4848,
    "longitude": -103.3659,
    "area": 4250.6,
    "forest": 0.4,
}

regionalization_method = "SP_IDW"

hydrograph, ensemble_hydrograph = regionalize(
    method=regionalization_method,
    model_config=model_config,
    model_name=model_structure,
    nash=nash,
    params=params,
    props=props,
    target_props=ungauged_props,
    min_NSE=0.7,
    size=5,
)

The `hydrograph` and `ensemble` outputs are netCDF files storing the time series. These files are opened by default using `xarray`, which provides convenient and powerful time series analysis and plotting tools. 

In [ ]:
display(hydrograph)

In [ ]:
display(ensemble_hydrograph)

In [ ]:
qq = ensemble_hydrograph.q_sim[0, :, 0]

ensemble_hydrograph.q_sim[:, :, 0].plot.line("b", x="time", add_legend=False)
ensemble_hydrograph.q_sim[1, :, 0].plot.line(
    "b", x="time", label="Regionalized hydrographs"
)
qq.plot.line("r", x="time", label="observations")
plt.legend(loc="lower left")
plt.show()

In [ ]:
print("Max: ", hydrograph.max())
print("Mean: ", hydrograph.mean())
print("Monthly means: ", hydrograph.groupby("time.month").mean(dim="time"))